In [6]:
from sympy import primerange, sqrt
from decimal import Decimal, getcontext
from math import isclose

# Set precision high enough for sqrt of large numbers
getcontext().prec = 150

# Generate the list of 100 large integers: (10^100) * sqrt(p_i)
def generate_large_numbers():
    primes = list(primerange(1, 550))[:100]  # First 100 primes
    scale = Decimal(10) ** 100
    items = [int(scale * Decimal(str(sqrt(p).evalf()))) for p in primes]  # Convert Float to Decimal via str
    return items

# FPTAS implementation
def approx_subset_sum(items, target, eps):
    items = sorted(items)
    n = len(items)
    L = [0]
    for x in items:
        new_sums = [s + x for s in L]
        U = sorted(set(L + new_sums))
        trimmed = []
        last = U[0]
        trimmed.append(last)
        for z in U[1:]:
            if z > last * (1 + eps / n):
                trimmed.append(z)
                last = z
        L = [s for s in trimmed if s <= target]
    return L[-1] if L else 0

# Generate the 100-item list
items = generate_large_numbers()

# Example usage — you define the target!
target = 7 * 113596441 * 10**94  # Example target
epsilon = 0.0001  # e.g. 1% error tolerance

# Measure runtime
import time
start_time = time.time()

# Compute the approximate subset sum
result = approx_subset_sum(items, target, epsilon)

# Measure runtime
end_time = time.time()
runtime = end_time - start_time

# Calculate the error and its number of digits
error = abs(target - result)
num_digits_in_error = len(str(error))

# Print the results
print(f"Approximate subset sum (ε={epsilon}):\n{result}")
print(f"Target sum:\n{target}")
print(f"Error (difference):\n{error}")
print(f"Number of digits in the error:\n{num_digits_in_error}")
print(f"Time taken: {runtime:.2f} seconds")

Approximate subset sum (ε=0.0001):
7951740972292899200000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Target sum:
7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Error (difference):
9897707100800000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Number of digits in the error:
97
Time taken: 539.25 seconds


In [ ]:
# worse
def subset_sum_alternate(M: list[int], S: int) -> list[int]:
    """
    M is a list (length 100) of integers and S is the target sum.
    Return the subset that sums the closest to S.
    """
    T = [0] * len(M) # binary list to show if the element from M is in the subset

    # Choose the first ele, then the last, then the second, then the second to last, etc.
    # This is a bit of a hack, but it works for this problem.
    for i in range(len(M) // 2):
        if sum(M[j] for j in range(len(M)) if T[j]) + M[i] <= S:
            T[i] = 1
        if sum(M[j] for j in range(len(M)) if T[j]) + M[len(M) - 1 - i] <= S:
            T[len(M) - 1 - i] = 1

    return T


def subset_sum_refine_3(M: list[int], S: int, subset_sum_function: Callable, k_l_pairs: List[tuple], N=None) -> list[int]:
    """
    M is a list (length 100) of integers and S is the target sum.
    Return the subset that sums the closest to S.
    Calls a different function to start this process, then refines the result.
    k_l_pairs: A list of (k, l) pairs to use for each iteration.
    N: Number of iterations (optional). If not provided, defaults to the length of k_l_pairs.
    """

    if N is None:
        N = len(k_l_pairs)

    print(f"Refining with {N} iterations and varying k, l pairs", flush=True)
    T0 = subset_sum_function(M, S)
    sum0 = sum_from_bin_list(T0, M)
    err = abs(S - sum0)

    for i in range(N):
        k, l = k_l_pairs[i % len(k_l_pairs)]  # Cycle through k_l_pairs if N > len(k_l_pairs)
        print(f"Iteration {i + 1} of {N}: Using k = {k}, l = {l}", flush=True)
        improve = True
        while improve:
            improve = False
            # Generate all combinations of k elements to remove
            remove_indices = [i for i in range(len(M)) if T0[i] == 1]
            add_indices = [i for i in range(len(M)) if T0[i] == 0]

            from itertools import combinations

            for remove_comb in combinations(remove_indices, k):
                for add_comb in combinations(add_indices, l):
                    # Calculate the new sum after removing and adding elements
                    new_sum = sum0
                    for r in remove_comb:
                        new_sum -= M[r]
                    for a in add_comb:
                        new_sum += M[a]

                    new_err = abs(S - new_sum)
                    if new_err < err:
                        # Update the subset and the error
                        print(f"Improving: removing {remove_comb} and adding {add_comb}", flush=True)
                        for r in remove_comb:
                            T0[r] = 0
                        for a in add_comb:
                            T0[a] = 1
                        sum0 = new_sum
                        err = new_err
                        improve = True
                        break
                if improve:
                    break

    return T0

def timer(func: Callable) -> Callable:
    """
    Decorator to time a function.
    """
    def wrapper(*args, **kwargs):
        import time
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Time taken: {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timer
def main(subset_sum_function=subset_sum_backwards, ID=113596441, extra_args=[]):
    print(f"Using {subset_sum_function.__name__} with ID {ID}")
    print()
    # generate the first 100 prime numbers
    primes = []
    i = 2
    while len(primes) < 100:
        if is_prime(i):
            primes.append(i)
        i += 1

    # print(len(primes))
    # print(primes)
    # print()

    # M = [10**100 * i**0.5 for i in primes]
    M = [math.floor(Decimal(i).sqrt() * Decimal(10**100)) for i in primes]
    # print(len(M))
    # print(M)

    S = 7 * ID * 10**94

    print("Number of digits in the target sum:", len(str(int(S))))
    print("Target sum:", S)
    print()

    exp_target = sum_from_bin_list(subset_sum_function(M, S, *extra_args), M)
    print("Number of digits in the subset sum:", len(str(int(exp_target))))
    print("Subset sum:", exp_target)
    print()

    print("Number of digits in the difference:", len(str(int(S - exp_target))))
    print("Difference:", S - exp_target) 
    print()

main(subset_sum_function=subset_sum_refine_3, ID=ID, extra_args=[subset_sum_alternate, [(1, 1), (1, 2), (2, 3), (3, 3), (3, 2), (2, 1), (1, 1)]])


In [ ]:
def check_binary_list(binary_list, M=None, target=None):
    """
    Check how close the sum of a binary list is to the target.
    
    Args:
        binary_list: A list of 0s and 1s indicating which elements to include
        M: The list of integers (uses the already generated one if None)
        target: The target sum (uses the already calculated one if None)
        
    Returns:
        Dictionary containing sum, error, and other information
    """
    import time
    
    if M is None:
        try:
            # Try to use the M from the notebook
            M = globals().get('M') or generate_large_numbers()
        except:
            print("Error: Could not find or generate M list")
            return
    
    if target is None:
        try:
            # Try to use the target from the notebook
            target = globals().get('target') or (7 * 113596441 * 10**94)
        except:
            print("Error: Could not find or generate target")
            return
    
    # Validate binary list
    if len(binary_list) != len(M):
        print(f"Error: Binary list length ({len(binary_list)}) must match M length ({len(M)})")
        return
    
    if not all(b in [0, 1] for b in binary_list):
        print("Error: Input must be a binary list (containing only 0s and 1s)")
        return
    
    # Calculate the sum
    start_time = time.time()
    subset_sum = sum(M[i] for i in range(len(M)) if binary_list[i])
    end_time = time.time()
    
    # Calculate the error
    error = abs(target - subset_sum)
    
    # Prepare results
    result = {
        "subset_sum": subset_sum,
        "target": target,
        "error": error,
        "error_digits": len(str(error)),
        "time": end_time - start_time,
        "num_elements": sum(binary_list),
        "percentage_error": (error / target) * 100 if target != 0 else float('inf')
    }
    
    # Print the results
    print(f"Subset sum:\n{subset_sum}")
    print(f"Target sum:\n{target}")
    print(f"Error (difference):\n{error}")
    print(f"Number of digits in error: {result['error_digits']}")
    print(f"Percentage error: {result['percentage_error']:.8f}%")
    print(f"Number of elements in subset: {result['num_elements']} out of {len(M)}")
    print(f"Time taken: {result['time']:.6f} seconds")
    
    return result

import math
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

primes = []
i = 2
while len(primes) < 100:
    if is_prime(i):
        primes.append(i)
    i += 1

    # print(len(primes))
    # print(primes)
    # print()

# M = [10**100 * i**0.5 for i in primes]
M = [math.floor(Decimal(i).sqrt() * Decimal(10**100)) for i in primes]
# print(len(M))
# print(M)

ID = 113596441
S = 7 * ID * 10**94
# Example usage
# 92 digits of error
# binary_list = [0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
#                0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
#                0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
#                0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
#                1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
#                1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
#                0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
#                1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
#                0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
#                1, 1, 1, 0, 1, 1, 1, 0, 1, 1,] 
binary_list = [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0, 0, 0, 0,]

check_binary_list(binary_list, M, S)

Subset sum:
0
Target sum:
7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Error (difference):
7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Number of digits in error: 103
Percentage error: 100.00000000%
Number of elements in subset: 0 out of 100
Time taken: 0.000000 seconds


{'subset_sum': 0,
 'target': 7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,
 'error': 7951750870000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000,
 'error_digits': 103,
 'time': 0.0,
 'num_elements': 0,
 'percentage_error': 100.0}